In [1]:
#langpair_list = 'ende enes enfr enit enja enko enpt enzh enzt'.split(' ')
#langpair_list = 'enes enzh'.split(' ')
langpair_list = 'ende enes enit enko enpt'.split(' ') # enzh enzt enfr enja still in progress

In [2]:
# noexport

import msgpack
import rocksdb
import glob
import os
import numpy as np
from plotly_utils import plot_histogram

# langpair_to_cache_db = {}
# langpair_to_translation_db = {}
# for langpair_arg in langpair_list:
#   translation_cache_db = rocksdb.DB('translation_cache/' + langpair_arg, rocksdb.Options(create_if_missing=True))
#   cache_db = rocksdb.DB('simulation_cache/' + langpair_arg, rocksdb.Options(create_if_missing=True))
#   langpair_to_cache_db[langpair_arg] = cache_db
#   langpair_to_translation_db[langpair_arg] = translation_cache_db


In [3]:
import glob

source_file_to_google = {}
source_files = glob.glob('/Users/geza/intel_evaluation/GCP_translate_test/**/**/**/source.txt')
for source_file in source_files:
  google_translate_file = glob.glob(source_file.replace('source.txt', 'results/*.txt'))[0]
  source_file_to_google[source_file] = google_translate_file

In [4]:
# #it = langpair_to_cache_db['enes'].iteritems()
# #it.seek_to_first()
# #while True:
# #  next(it)
# for x in langpair_to_cache_db['enes'].iteritems():
#   print(x)

In [4]:
# noexport

for langpair_arg in langpair_list:
  if os.path.exists('simulation_output_list/' + langpair_arg + '.msgpack') and os.path.exists('translation_dict/' + langpair_arg + '.msgpack'):
    continue
  output_list = []
  #cache_db = langpair_to_cache_db[langpair_arg]
  cache_db = rocksdb.DB('simulation_cache/' + langpair_arg, rocksdb.Options(create_if_missing=True))
  source_files = glob.glob('/Users/geza/intel_evaluation/GCP_translate_test/**/**/**/source.txt')
  target_files = glob.glob('/Users/geza/intel_evaluation/GCP_translate_test/**/**/**/target.txt')
  target_files_set = set(target_files)
  for source_file in source_files:
    target_file = source_file.replace('source.txt', 'target.txt')
    if target_file not in target_files_set:
      print('missing target file for: ' + source_file)
      continue
    google_translate_file = source_file_to_google[source_file]
    google_translate_lines = open(google_translate_file).readlines()
    source_lines = open(source_file).readlines()
    target_lines = open(target_file).readlines()
    if len(source_lines) != len(target_lines):
      print('source line length', len(source_lines), 'does not match target line length', len(target_lines))
      continue
    source_lines = [x.strip() for x in source_lines]
    target_lines = [x.strip() for x in target_lines]
    langpair = source_file.replace('/Users/geza/intel_evaluation/GCP_translate_test/', '').split('/', 1)[0]
    if langpair != langpair_arg:
      continue
    #print(langpair)
    src_lang = langpair[0:2]
    trg_lang = langpair[2:]
    for src_sentence,trg_sentence,google_translate in zip(source_lines, target_lines, google_translate_lines):
      key = msgpack.dumps([src_sentence, trg_sentence])
      cached_result = cache_db.get(key)
      if cached_result is not None:
        output = msgpack.loads(cached_result, raw=False, strict_map_key=False)
        output_list.append([src_sentence, trg_sentence, output, google_translate])

  msgpack.dump(output_list, open('simulation_output_list/' + langpair_arg + '.msgpack', 'wb'))
  
  translation_cache_db = rocksdb.DB('translation_cache/' + langpair_arg, rocksdb.Options(create_if_missing=True))
  translation_dict = {}
  for src_sentence,trg_sentence,output,google_translate in output_list:
    key = msgpack.dumps(src_sentence)
    cached_result = translation_cache_db.get(key)
    if cached_result is not None:
      word_list,delimiter_list = msgpack.loads(cached_result)
      translation_dict[src_sentence] = [word_list, delimiter_list]

  msgpack.dump(translation_dict, open('translation_dict/' + langpair_arg + '.msgpack', 'wb'))

missing target file for: /Users/geza/intel_evaluation/GCP_translate_test/ende/SYNDICATION_TM/test/source.txt
missing target file for: /Users/geza/intel_evaluation/GCP_translate_test/ende/SYNDICATION_TM/test/source.txt
missing target file for: /Users/geza/intel_evaluation/GCP_translate_test/ende/SYNDICATION_TM/test/source.txt
missing target file for: /Users/geza/intel_evaluation/GCP_translate_test/ende/SYNDICATION_TM/test/source.txt
missing target file for: /Users/geza/intel_evaluation/GCP_translate_test/ende/SYNDICATION_TM/test/source.txt


In [6]:
# noexport

langpair_to_oracle_accuracy = {}
for langpair_arg in langpair_list:
  words_correct = 0
  words_total = 0
  print(langpair_arg)
  #cache_db = langpair_to_cache_db[langpair_arg]
  #translation_cache_db = langpair_to_translation_db[langpair_arg]
  translation_dict = msgpack.load(open('translation_dict/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  output_list = msgpack.load(open('simulation_output_list/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  for src_sentence,trg_sentence,output,google_translate in output_list:
    #key = msgpack.dumps(src_sentence)
    word_list,delimiter_list = translation_dict[src_sentence]
    #cached_result = translation_cache_db.get(key)
    word_set = set(word_list)
    for word_info in output:
      word_ref = word_info[2]
      if word_ref in word_set:
        words_correct += 1
      words_total += 1
  print(words_total)
  print(words_correct)
  langpair_to_oracle_accuracy[langpair_arg] = words_correct / words_total


enes
154202
105931
enzh
191928
123947


In [7]:
import json

synonym_dicts = {} # (srclang, trglang) -> synonym dict

def lookup_synonyms(word, srclang, trglang, limit_num_synonyms=None):
  lpair = (srclang, trglang)
  syndict = synonym_dicts.get(lpair, None)
  if syndict is not None:
    output = syndict.get(word, [])
    if limit_num_synonyms is None:
      return output
    else:
      return output[:limit_num_synonyms]
  syndict = json.load(open(f'/Users/geza/panlex-20190701-json/language_pairs/{srclang}_{trglang}.json', 'rt'))
  synonym_dicts[lpair] = syndict
  output = syndict.get(word, [])
  if limit_num_synonyms is None:
    return output
  else:
    return output[:limit_num_synonyms]

In [8]:
#lookup_synonyms('convenient', 'en', 'ja', 3)

In [9]:
from ling.segmenter import get_segmenter

segmenter_cached = {}

def get_segmenter_cached(lang):
  if lang in segmenter_cached:
    return segmenter_cached[lang]
  segmenter_cached[lang] = get_segmenter(lang)
  return segmenter_cached[lang]

def segment_sentence(sentence, lang):
  # returned object has 2 fields of interest:
  # .tokens
  # .delimiters
  return get_segmenter_cached(lang).segment(sentence)


In [10]:
# segment_sentence('hello world', 'en').tokens

In [11]:
# noexport

langpair_to_num_src_trg_synonyms_to_accuracy = {}
for langpair_arg in langpair_list:
  translation_dict = msgpack.load(open('translation_dict/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  output_list = msgpack.load(open('simulation_output_list/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  langpair_to_num_src_trg_synonyms_to_accuracy[langpair_arg] = {}
  for num_synonyms_sourcelang in range(6):
    for num_synonyms_targetlang in range(6):
      if (num_synonyms_sourcelang, num_synonyms_targetlang) in langpair_to_num_src_trg_synonyms_to_accuracy[langpair_arg]:
        continue
      src_lang = langpair_arg[0:2]
      trg_lang = langpair_arg[2:]
      print(langpair_arg, num_synonyms_sourcelang, num_synonyms_targetlang)
      words_correct = 0
      words_total = 0
      #cache_db = langpair_to_cache_db[langpair_arg]
      #translation_cache_db = langpair_to_translation_db[langpair_arg]
      for src_sentence,trg_sentence,output,google_translate in output_list:
        #key = msgpack.dumps(src_sentence)
        word_list,delimiter_list = translation_dict[src_sentence]
        #cached_result = translation_cache_db.get(key)
        word_set = set(word_list)
        for word in word_list:
          for synonym in lookup_synonyms(word, trg_lang, trg_lang, num_synonyms_targetlang): # target to target synonyms
            word_set.add(synonym)
        for word in segment_sentence(src_sentence, src_lang).tokens:
          for synonym in lookup_synonyms(word, src_lang, trg_lang, num_synonyms_sourcelang): # source to target synonyms
            word_set.add(synonym)
        for word_info in output:
          word_ref = word_info[2]
          if word_ref in word_set:
            words_correct += 1
          words_total += 1
      print(words_correct / words_total)
      langpair_to_num_src_trg_synonyms_to_accuracy[langpair_arg][(num_synonyms_sourcelang, num_synonyms_targetlang)] = words_correct / words_total



enes 0 0
0.6869625556088766
enes 0 1
0.6871117106133513
enes 0 2
0.7031361460940844
enes 0 3
0.7130322563909677
enes 0 4
0.7179803115394093
enes 0 5
0.723589836707695
enes 1 0
0.701787266053618
enes 1 1
0.7018456310553689
enes 1 2
0.7136677864100336
enes 1 3
0.721631366648941
enes 1 4
0.7262551717876552
enes 1 5
0.7313069869392096
enes 2 0
0.7134797214043916
enes 2 1
0.7135251164057534
enes 2 2
0.7212163266364898
enes 2 3
0.7277467218324016
enes 2 4
0.7314626269438788
enes 2 5
0.7363198920895968
enes 3 0
0.7212357816370735
enes 3 1
0.7212811766384353
enes 3 2
0.727856966835709
enes 3 3
0.7330579369917382
enes 3 4
0.7358724270761728
enes 3 5
0.7389592871687786
enes 4 0
0.7260735917822078
enes 4 1
0.7261189867835696
enes 4 2
0.7315404469462135
enes 4 3
0.7358464870753946
enes 4 4
0.7377206521316195
enes 4 5
0.7406842972205289
enes 5 0
0.7288880818666424
enes 5 1
0.7289334768680044
enes 5 2
0.7339723220191696
enes 5 3
0.7372148221164446
enes 5 4
0.7388490421654713
enes 5 5
0.7416765022502

In [12]:
# noexport

langpair_to_num_src_trg_synonyms_googtrans_to_accuracy = {}
for langpair_arg in langpair_list:
  translation_dict = msgpack.load(open('translation_dict/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  output_list = msgpack.load(open('simulation_output_list/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  langpair_to_num_src_trg_synonyms_googtrans_to_accuracy[langpair_arg] = {}
  for num_synonyms_sourcelang in range(6):
    for num_synonyms_targetlang in range(6):
      if (num_synonyms_sourcelang, num_synonyms_targetlang) in langpair_to_num_src_trg_synonyms_googtrans_to_accuracy[langpair_arg]:
        continue
      src_lang = langpair_arg[0:2]
      trg_lang = langpair_arg[2:]
      print(langpair_arg, num_synonyms_sourcelang, num_synonyms_targetlang)
      words_correct = 0
      words_total = 0
      #cache_db = langpair_to_cache_db[langpair_arg]
      #translation_cache_db = langpair_to_translation_db[langpair_arg]
      for src_sentence,trg_sentence,output,google_translate in output_list:
        #key = msgpack.dumps(src_sentence)
        segmented_google_translate = segment_sentence(google_translate, trg_lang)
        word_list = segmented_google_translate.tokens
        delimiter_list = segmented_google_translate.delimiters
        #cached_result = translation_cache_db.get(key)
        word_set = set(word_list)
        for word in word_list:
          for synonym in lookup_synonyms(word, trg_lang, trg_lang, num_synonyms_targetlang): # target to target synonyms
            word_set.add(synonym)
        for word in segment_sentence(src_sentence, src_lang).tokens:
          for synonym in lookup_synonyms(word, src_lang, trg_lang, num_synonyms_sourcelang): # source to target synonyms
            word_set.add(synonym)
        for word_info in output:
          word_ref = word_info[2]
          if word_ref in word_set:
            words_correct += 1
          words_total += 1
      print(words_correct / words_total)
      langpair_to_num_src_trg_synonyms_googtrans_to_accuracy[langpair_arg][(num_synonyms_sourcelang, num_synonyms_targetlang)] = words_correct / words_total



enes 0 0
0.7166249464987484
enes 0 1
0.7166444014993321
enes 0 2
0.7342446920273408
enes 0 3
0.7433301772999054
enes 0 4
0.7482976874489307
enes 0 5
0.7526102125783064
enes 1 0
0.7334405520032166
enes 1 1
0.7334600070038002
enes 1 2
0.7455999273679979
enes 1 3
0.7526750625802519
enes 1 4
0.7571626827148805
enes 1 5
0.7610342278310268
enes 2 0
0.7445688123370644
enes 2 1
0.744588267337648
enes 2 2
0.7526815475804465
enes 2 3
0.7584078027522341
enes 2 4
0.7619097028572911
enes 2 5
0.7656839729705192
enes 3 0
0.751449397543482
enes 3 1
0.7514688525440656
enes 3 2
0.7583818627514559
enes 3 3
0.7628694828860845
enes 3 4
0.7655931829677954
enes 3 5
0.768323368049701
enes 4 0
0.7553468826604065
enes 4 1
0.7553663376609902
enes 4 2
0.7612158078364742
enes 4 3
0.7651132929533988
enes 4 4
0.7671106730133201
enes 4 5
0.7697306130919184
enes 5 0
0.7578630627358919
enes 5 1
0.7578825177364755
enes 5 2
0.763394767901843
enes 5 3
0.7664427179932816
enes 5 4
0.7682131230463937
enes 5 5
0.7707163331214

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/qf/y_gq4r991hd8shhm09zd0jpw0000gn/T/jieba.cache


enzh 0 0


Loading model cost 0.763 seconds.
Prefix dict has been built successfully.


0.7740454753866033
enzh 0 1
0.7740663165353674
enzh 0 2
0.782684131549331
enzh 0 3
0.7883529240131716
enzh 0 4
0.7916145637947564
enzh 0 5
0.7949283064482514
enzh 1 0
0.786680421824851
enzh 1 1
0.7867012629736151
enzh 1 2
0.7932193322495936
enzh 1 3
0.7981170022091618
enzh 1 4
0.8010503938977116
enzh 1 5
0.8041036221916552
enzh 2 0
0.7914582551790255
enzh 2 1
0.7914790963277896
enzh 2 2
0.7969915801758993
enzh 2 3
0.8014776374473761
enzh 2 4
0.8042859822433412
enzh 2 5
0.8072193739318911
enzh 3 0
0.7945219040473511
enzh 3 1
0.7945427451961152
enzh 3 2
0.79972698095119
enzh 3 3
0.8037649535242383
enzh 3 4
0.8064899337251469
enzh 3 5
0.8092409653620107
enzh 4 0
0.7968248509857864
enzh 4 1
0.7968404818473594
enzh 4 2
0.8018736192738943
enzh 4 3
0.8054374557125589
enzh 4 4
0.8080842816056021
enzh 4 5
0.8104705931390939
enzh 5 0
0.7986328206410738
enzh 5 1
0.7986484515026469
enzh 5 2
0.803546121462215
enzh 5 3
0.8070318035930141
enzh 5 4
0.8095327414447084
enzh 5 5
0.8118825809678629


In [13]:
langpair_to_num_src_trg_synonyms_prefix_nextword_to_accuracy = {}

for langpair_arg in langpair_list:
  langpair_to_num_src_trg_synonyms_prefix_nextword_to_accuracy[langpair_arg] = {}
  translation_dict = msgpack.load(open('translation_dict/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  output_list = msgpack.load(open('simulation_output_list/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  for num_synonyms_sourcelang in range(6):
    for num_synonyms_targetlang in range(6):
      if (num_synonyms_sourcelang, num_synonyms_targetlang) in langpair_to_num_src_trg_synonyms_prefix_nextword_to_accuracy[langpair_arg]:
        continue
      src_lang = langpair_arg[0:2]
      trg_lang = langpair_arg[2:]
      print(langpair_arg, num_synonyms_sourcelang, num_synonyms_targetlang)
      words_correct = 0
      words_total = 0
      #cache_db = langpair_to_cache_db[langpair_arg]
      #translation_cache_db = langpair_to_translation_db[langpair_arg]
      for src_sentence,trg_sentence,output,google_translate in output_list:
        word_set = set()
        for word in segment_sentence(src_sentence, src_lang).tokens:
          for synonym in lookup_synonyms(word, src_lang, trg_lang, num_synonyms_sourcelang): # source to target synonyms
            word_set.add(synonym)
        for word_prediction_info in output:
          [elapsed_time, is_mt_recompute_needed, next_actual_word, next_predicted_word, is_next_actual_word_anywhere_in_suffix] = word_prediction_info
          words_total += 1
          is_correct = False
          if next_actual_word == next_predicted_word:
            is_correct = True
          if next_actual_word in word_set:
            is_correct = True
          if not is_correct:
            for synonym in lookup_synonyms(next_predicted_word, trg_lang, trg_lang, num_synonyms_targetlang): # target to target synonyms
              if synonym == next_actual_word:
                is_correct = True
          if is_correct:
            words_correct += 1
      print(words_correct / words_total)
      langpair_to_num_src_trg_synonyms_prefix_nextword_to_accuracy[langpair_arg][(num_synonyms_sourcelang, num_synonyms_targetlang)] = words_correct / words_total


enes 0 0
0.6845436505363095
enes 0 1
0.6845890455376714
enes 0 2
0.6942711508281345
enes 0 3
0.699193265975798
enes 0 4
0.7015408360462251
enes 0 5
0.7044785411343563
enes 1 0
0.7151333964540019
enes 1 1
0.7151723064551692
enes 1 2
0.7219945266598358
enes 1 3
0.7257752817732585
enes 1 4
0.7278764218362926
enes 1 5
0.730450966913529
enes 2 0
0.7309957069298713
enes 2 1
0.7310346169310386
enes 2 2
0.7347764620432938
enes 2 3
0.7378698071360942
enes 2 4
0.7396531821895954
enes 2 5
0.7420266922608008
enes 3 0
0.7417478372524351
enes 3 1
0.7417867472536024
enes 3 2
0.7449903373497101
enes 3 3
0.7474740924242228
enes 3 4
0.7489461874683856
enes 3 5
0.7507749575232487
enes 4 0
0.7469034124071023
enes 4 1
0.7469423224082696
enes 4 2
0.7496400824892024
enes 4 3
0.7517995875539877
enes 4 4
0.7527010025810301
enes 4 5
0.7544908626347259
enes 5 0
0.7495622624868679
enes 5 1
0.7496011724880352
enes 5 2
0.7522081425662442
enes 5 3
0.7539461226183837
enes 5 4
0.7547178376415351
enes 5 5
0.75642987769

In [14]:
langpair_to_num_src_trg_synonyms_prefix_oracle_to_accuracy = {}

for langpair_arg in langpair_list:
  langpair_to_num_src_trg_synonyms_prefix_oracle_to_accuracy[langpair_arg] = {}
  translation_dict = msgpack.load(open('translation_dict/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  output_list = msgpack.load(open('simulation_output_list/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  for num_synonyms_sourcelang in range(6):
    for num_synonyms_targetlang in range(6):
      if (num_synonyms_sourcelang, num_synonyms_targetlang) in langpair_to_num_src_trg_synonyms_prefix_oracle_to_accuracy[langpair_arg]:
        continue
      src_lang = langpair_arg[0:2]
      trg_lang = langpair_arg[2:]
      print(langpair_arg, num_synonyms_sourcelang, num_synonyms_targetlang)
      words_correct = 0
      words_total = 0
      #cache_db = langpair_to_cache_db[langpair_arg]
      #translation_cache_db = langpair_to_translation_db[langpair_arg]
      for src_sentence,trg_sentence,output,google_translate in output_list:
        word_set = set()
        for word in segment_sentence(src_sentence, src_lang).tokens:
          for synonym in lookup_synonyms(word, src_lang, trg_lang, num_synonyms_sourcelang): # source to target synonyms
            word_set.add(synonym)
        for word_prediction_info in output:
          [elapsed_time, is_mt_recompute_needed, next_actual_word, next_predicted_word, is_next_actual_word_anywhere_in_suffix] = word_prediction_info
          words_total += 1
          is_correct = False
          if is_next_actual_word_anywhere_in_suffix:
            is_correct = True
          if next_actual_word in word_set:
            is_correct = True
          if not is_correct:
            for synonym in lookup_synonyms(next_predicted_word, trg_lang, trg_lang, num_synonyms_targetlang): # target to target synonyms
              if synonym == next_actual_word:
                is_correct = True
          if is_correct:
            words_correct += 1
      print(words_correct / words_total)
      langpair_to_num_src_trg_synonyms_prefix_oracle_to_accuracy[langpair_arg][(num_synonyms_sourcelang, num_synonyms_targetlang)] = words_correct / words_total


enes 0 0
0.7321111269633338
enes 0 1
0.7321565219646956
enes 0 2
0.7407556322226689
enes 0 3
0.745178402355352
enes 0 4
0.7473314224199427
enes 0 5
0.7498800274964008
enes 1 0
0.7481939274458178
enes 1 1
0.7482328374469851
enes 1 2
0.7543287376298621
enes 1 3
0.7577982127339464
enes 1 4
0.7597242577917277
enes 1 5
0.7620523728615712
enes 2 0
0.759763167792895
enes 2 1
0.7598020777940623
enes 2 2
0.7632261578967847
enes 2 3
0.7661444079843323
enes 2 4
0.7677980830339425
enes 2 5
0.7699316480979495
enes 3 0
0.7672987380189621
enes 3 1
0.7673376480201295
enes 3 2
0.7703012931090388
enes 3 3
0.7726683181800496
enes 3 4
0.774017198220516
enes 3 5
0.7757357232720717
enes 4 0
0.7715788381473652
enes 4 1
0.7716177481485325
enes 4 2
0.7741209582236287
enes 4 3
0.7761831882854956
enes 4 4
0.7770586633117599
enes 4 5
0.7787382783621484
enes 5 0
0.7740366532210996
enes 5 1
0.7740755632222669
enes 5 2
0.776494468294834
enes 5 3
0.7781546283446389
enes 5 4
0.778900403367012
enes 5 5
0.78050219841506

In [15]:
langpair_to_num_src_trg_synonyms_prefix_oracle_allsyn_to_accuracy = {}

for langpair_arg in langpair_list:
  langpair_to_num_src_trg_synonyms_prefix_oracle_allsyn_to_accuracy[langpair_arg] = {}
  translation_dict = msgpack.load(open('translation_dict/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  output_list = msgpack.load(open('simulation_output_list/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  for num_synonyms_sourcelang in range(6):
    for num_synonyms_targetlang in range(6):
      if (num_synonyms_sourcelang, num_synonyms_targetlang) in langpair_to_num_src_trg_synonyms_prefix_oracle_allsyn_to_accuracy[langpair_arg]:
        continue
      src_lang = langpair_arg[0:2]
      trg_lang = langpair_arg[2:]
      print(langpair_arg, num_synonyms_sourcelang, num_synonyms_targetlang)
      words_correct = 0
      words_total = 0
      #cache_db = langpair_to_cache_db[langpair_arg]
      #translation_cache_db = langpair_to_translation_db[langpair_arg]
      for src_sentence,trg_sentence,output,google_translate in output_list:
        #word_list,delimiter_list = translation_dict[src_sentence]
        #cached_result = translation_cache_db.get(key)
        #word_set = set(word_list)
        word_set = set()
        #for word in word_list:
        #  for synonym in lookup_synonyms(word, trg_lang, trg_lang, num_synonyms_targetlang): # target to target synonyms
        #    word_set.add(synonym)
        for word in segment_sentence(src_sentence, src_lang).tokens:
          for synonym in lookup_synonyms(word, src_lang, trg_lang, num_synonyms_sourcelang): # source to target synonyms
            word_set.add(synonym)
        for word_prediction_info in output:
          [elapsed_time, is_mt_recompute_needed, next_actual_word, next_predicted_word, is_next_actual_word_anywhere_in_suffix] = word_prediction_info
          words_total += 1
          is_correct = False
          if is_next_actual_word_anywhere_in_suffix:
            is_correct = True
          if next_actual_word in word_set:
            is_correct = True
          if not is_correct:
            for synonym in lookup_synonyms(next_predicted_word, trg_lang, trg_lang, num_synonyms_targetlang): # target to target synonyms
              if synonym == next_actual_word:
                is_correct = True
          if is_correct:
            words_correct += 1
      print(words_correct / words_total)
      langpair_to_num_src_trg_synonyms_prefix_oracle_allsyn_to_accuracy[langpair_arg][(num_synonyms_sourcelang, num_synonyms_targetlang)] = words_correct / words_total


enes 0 0
0.7321111269633338
enes 0 1
0.7321565219646956
enes 0 2
0.7407556322226689
enes 0 3
0.745178402355352
enes 0 4
0.7473314224199427
enes 0 5
0.7498800274964008
enes 1 0
0.7481939274458178
enes 1 1
0.7482328374469851
enes 1 2
0.7543287376298621
enes 1 3
0.7577982127339464
enes 1 4
0.7597242577917277
enes 1 5
0.7620523728615712
enes 2 0
0.759763167792895
enes 2 1
0.7598020777940623
enes 2 2
0.7632261578967847
enes 2 3
0.7661444079843323
enes 2 4
0.7677980830339425
enes 2 5
0.7699316480979495
enes 3 0
0.7672987380189621
enes 3 1
0.7673376480201295
enes 3 2
0.7703012931090388
enes 3 3
0.7726683181800496
enes 3 4
0.774017198220516
enes 3 5
0.7757357232720717
enes 4 0
0.7715788381473652
enes 4 1
0.7716177481485325
enes 4 2
0.7741209582236287
enes 4 3
0.7761831882854956
enes 4 4
0.7770586633117599
enes 4 5
0.7787382783621484
enes 5 0
0.7740366532210996
enes 5 1
0.7740755632222669
enes 5 2
0.776494468294834
enes 5 3
0.7781546283446389
enes 5 4
0.778900403367012
enes 5 5
0.78050219841506

In [21]:
# noexport

langpair_to_num_src_trg_synonyms_nomt_to_accuracy = {}
for langpair_arg in langpair_list:
  translation_dict = msgpack.load(open('translation_dict/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  output_list = msgpack.load(open('simulation_output_list/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  langpair_to_num_src_trg_synonyms_nomt_to_accuracy[langpair_arg] = {}
  for num_synonyms_sourcelang in range(6):
    for num_synonyms_targetlang in range(1):
      if (num_synonyms_sourcelang, num_synonyms_targetlang) in langpair_to_num_src_trg_synonyms_nomt_to_accuracy[langpair_arg]:
        continue
      src_lang = langpair_arg[0:2]
      trg_lang = langpair_arg[2:]
      print(langpair_arg, num_synonyms_sourcelang, num_synonyms_targetlang)
      words_correct = 0
      words_total = 0
      #cache_db = langpair_to_cache_db[langpair_arg]
      #translation_cache_db = langpair_to_translation_db[langpair_arg]
      for src_sentence,trg_sentence,output,google_translate in output_list:
        word_set = set()
        for word in segment_sentence(src_sentence, src_lang).tokens:
          for synonym in lookup_synonyms(word, src_lang, trg_lang, num_synonyms_sourcelang): # source to target synonyms
            word_set.add(synonym)
        for word_info in output:
          word_ref = word_info[2]
          if word_ref in word_set:
            words_correct += 1
          words_total += 1
      print(words_correct / words_total)
      langpair_to_num_src_trg_synonyms_nomt_to_accuracy[langpair_arg][(num_synonyms_sourcelang, num_synonyms_targetlang)] = words_correct / words_total



enes 0 0
0.0
enes 1 0
0.2255353367660601
enes 2 0
0.29480810884424324
enes 3 0
0.34186327025589813
enes 4 0
0.358062800741884
enes 5 0
0.36474235094227053
enzh 0 0
0.0
enzh 1 0
0.12509899545662956
enzh 2 0
0.15524571714392899
enzh 3 0
0.17263765578758702
enzh 4 0
0.18219332249593598
enzh 5 0
0.18845087741236297


In [16]:
# noexport

lang_to_google_translate_accuracy_with_oracle = {}
lang_to_baseline_accuracy_with_oracle = {}
lang_to_baseline_accuracy = {}
lang_to_baseline_accuracy_fast = {}
for langpair_arg in langpair_list:
  print(langpair_arg)
  output_list = msgpack.load(open('simulation_output_list/' + langpair_arg + '.msgpack', 'rb'), raw=False, strict_map_key=False)
  frac_correct_predictions_list = []
  frac_fast_correct_predictions_list = []
  total_time_on_mt_needed_list = []
  word_prediction_time_needed_list = []
  total_num_correct_predictions_all = 0
  total_num_fast_correct_predictions_all = 0
  total_num_predictions_all = 0
  total_num_predictions_correct_anywhere_in_suffix = 0
  total_num_predictions_correct_google_translate = 0
  for src_sentence,trg_sentence,output,google_translate in output_list:
    num_total_predictions = len(output)
    if num_total_predictions == 0:
      continue
    num_correct_predictions = sum([1 for x in output if x[2] == x[3]])
    frac_correct_predictions = num_correct_predictions / num_total_predictions
    total_num_correct_predictions_all += num_correct_predictions
    total_num_predictions_all += num_total_predictions
    
    for word_prediction_info in output:
      [elapsed_time, is_mt_recompute_needed, next_actual_word, next_predicted_word, is_next_actual_word_anywhere_in_suffix] = word_prediction_info
      if is_next_actual_word_anywhere_in_suffix:
        total_num_predictions_correct_anywhere_in_suffix += 1
      if next_actual_word in google_translate:
        total_num_predictions_correct_google_translate += 1

    num_fast_correct_predictions = 0
    for x in [y for y in output if y[2] == y[3]]:
      mt_needed = x[1]
      if mt_needed and x[0] > 0.1: # too slow
        continue
      num_fast_correct_predictions += 1
    frac_fast_correct_predictions = num_fast_correct_predictions / num_total_predictions
    total_num_fast_correct_predictions_all += num_fast_correct_predictions

    frac_fast_correct_predictions_list.append(frac_fast_correct_predictions)
    frac_correct_predictions_list.append(frac_correct_predictions)
    total_time_on_mt_needed = sum([x[0] for x in output if x[1]])
    total_time_on_mt_needed_list.append(total_time_on_mt_needed)
    for word_prediction_time_needed in [x[0] for x in output if x[1]]:
      word_prediction_time_needed_list.append(word_prediction_time_needed)
  
  lang_to_google_translate_accuracy_with_oracle[langpair_arg] = total_num_predictions_correct_google_translate / total_num_predictions_all
  lang_to_baseline_accuracy_with_oracle[langpair_arg] = total_num_predictions_correct_anywhere_in_suffix / total_num_predictions_all
  lang_to_baseline_accuracy[langpair_arg] = total_num_correct_predictions_all / total_num_predictions_all
  lang_to_baseline_accuracy_fast[langpair_arg] = total_num_fast_correct_predictions_all / total_num_predictions_all
  print('num samples for frac_correct_predictions_list', len(frac_correct_predictions_list))
  print('mean over predictions word-weighted', total_num_correct_predictions_all / total_num_predictions_all)
  print('mean over fast predictions word-weighted', total_num_fast_correct_predictions_all / total_num_predictions_all)
  print('median for frac_correct_predictions_list', np.median(frac_correct_predictions_list))
  #plot_histogram(frac_correct_predictions_list)

enes
num samples for frac_correct_predictions_list 10593
mean over predictions word-weighted 0.6845436505363095
mean over fast predictions word-weighted 0.5288777058663312
median for frac_correct_predictions_list 0.6666666666666666
enzh
num samples for frac_correct_predictions_list 11425
mean over predictions word-weighted 0.5891375932641407
mean over fast predictions word-weighted 0.4474855154016089
median for frac_correct_predictions_list 0.5625


In [22]:
# noexport

import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='no MT, 5 src syn for each word', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_nomt_to_accuracy[lang][(5, 0)] for lang in langpair_list]),
    go.Bar(name='prefix-constrained MT', x=langpair_list, y=[lang_to_baseline_accuracy[lang] for lang in langpair_list]),
    go.Bar(name='prefix-constrained MT, plus 5 src syn for each word', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_prefix_nextword_to_accuracy[lang][(5,0)] for lang in langpair_list]),
    go.Bar(name='prefix-constrained MT, plus 5 trg syn for *next* word', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_prefix_nextword_to_accuracy[lang][(0,5)] for lang in langpair_list]),
    go.Bar(name='prefix-constrained MT, plus 5 src syn for each word + 5 trg syn for *next* word', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_prefix_oracle_allsyn_to_accuracy[lang][(5,5)] for lang in langpair_list]),
    go.Bar(name='prefix-constrained MT with oracle', x=langpair_list, y=[lang_to_baseline_accuracy_with_oracle[lang] for lang in langpair_list]),
    go.Bar(name='prefix-constrained MT with oracle, plus 5 src syn for each word', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_prefix_oracle_to_accuracy[lang][(5,0)] for lang in langpair_list]),
    go.Bar(name='prefix-constrained MT with oracle, plus 5 trg syn for *next* word', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_prefix_oracle_to_accuracy[lang][(0,5)] for lang in langpair_list]),
    go.Bar(name='prefix-constrained MT with oracle, plus 5 src syn for each word + 5 trg syn for *next* word', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_prefix_oracle_allsyn_to_accuracy[lang][(5,5)] for lang in langpair_list]),
    go.Bar(name='fixed MT with oracle', x=langpair_list, y=[langpair_to_oracle_accuracy[lang] for lang in langpair_list]),
    go.Bar(name='fixed MT with oracle, plus 5 src syn for each word', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_to_accuracy[lang][(5,0)] for lang in langpair_list]),
    go.Bar(name='fixed MT with oracle, plus 5 trg syn for each word', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_to_accuracy[lang][(0,5)] for lang in langpair_list]),
    go.Bar(name='fixed MT with oracle, plus 5 src syn + 5 trg syn for each word', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_to_accuracy[lang][(5,5)] for lang in langpair_list]),
    #go.Bar(name='google translate with oracle', x=langpair_list, y=[lang_to_google_translate_accuracy_with_oracle[lang] for lang in langpair_list]),
    go.Bar(name='google translate with oracle', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_googtrans_to_accuracy[lang][(0,0)] for lang in langpair_list]),
    go.Bar(name='google translate with oracle, plus 5 src syn for each word', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_googtrans_to_accuracy[lang][(5,0)] for lang in langpair_list]),
    go.Bar(name='google translate with oracle, plus 5 trg syn for each word', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_googtrans_to_accuracy[lang][(0,5)] for lang in langpair_list]),
    go.Bar(name='google translate with oracle, plus 5 src syn + 5 trg syn for each word', x=langpair_list, y=[langpair_to_num_src_trg_synonyms_googtrans_to_accuracy[lang][(5,5)] for lang in langpair_list]),
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [18]:
datalist = []
for num_synonyms_sourcelang in range(1):
  for num_synonyms_targetlang in range(6):
    yvals = [langpair_to_num_src_trg_synonyms_prefix_nextword_to_accuracy[lang][(num_synonyms_sourcelang, num_synonyms_targetlang)] for lang in langpair_list]
    dataitem = go.Bar(name=f'{num_synonyms_sourcelang} src {num_synonyms_targetlang} trg syn', x=langpair_list, y=yvals)
    datalist.append(dataitem)
fig = go.Figure(data=datalist)
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [19]:
datalist = []
for num_synonyms_sourcelang in range(6):
  for num_synonyms_targetlang in range(6):
    yvals = [langpair_to_num_src_trg_synonyms_to_accuracy[lang][(num_synonyms_sourcelang, num_synonyms_targetlang)] for lang in langpair_list]
    dataitem = go.Bar(name=f'{num_synonyms_sourcelang} src {num_synonyms_targetlang} trg syn', x=langpair_list, y=yvals)
    datalist.append(dataitem)
fig = go.Figure(data=datalist)
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [20]:
datalist = []
for num_synonyms_sourcelang in range(6):
  for num_synonyms_targetlang in range(6):
    yvals = [langpair_to_num_src_trg_synonyms_googtrans_to_accuracy[lang][(num_synonyms_sourcelang, num_synonyms_targetlang)] for lang in langpair_list]
    dataitem = go.Bar(name=f'{num_synonyms_sourcelang} src {num_synonyms_targetlang} trg syn', x=langpair_list, y=yvals)
    datalist.append(dataitem)
fig = go.Figure(data=datalist)
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()